In [133]:
import numpy as np
import nbimporter
import benchmarks

class LinearCombinationSwarm:
    def __init__(self, objective_function, dimension, K_best=10, pl=0.3, pb=0.5, mu=0.5, sigma=0.1, beta_best=0.1, beta_h=0.2):
        """
        Initialize the algorithm parameters.
        :param objective_function: The function to optimize.
        :param dimension: The dimensionality of the search space.
        :param K_best: Number of best trials to keep track of.
        :param pl: Probability of choosing the linear combination sampling strategy.
        :param pb: Probability of choosing the ball sampling strategy (if linear combination not chosen).
        :param mu: Mean of the normal distribution for linear combination weight alpha.
        :param sigma: Standard deviation of the normal distribution for alpha.
        :param beta_best: Proportion of best trials to sample for linear combination.
        :param beta_h: Proportion of history trials to include in linear combination sampling.
        """
        self.objective_function = objective_function
        self.dimension = dimension
        self.K_best = K_best
        self.pl = pl
        self.pb = pb
        self.mu = mu
        self.sigma = sigma
        self.beta_best = beta_best
        self.beta_h = beta_h
        
        # Initialize history and best trials sets
        self.H = []  # History of all sampled points
        self.S = []  # Best K_best trials

    def evaluate(self, x):
        """Evaluate the objective function."""
        return self.objective_function(x)
    
    def sample_random(self):
        """Sample a point uniformly at random in the search space [-5, 5]^dimension."""
        return np.random.uniform(-5, 5, self.dimension)
    
    def linear_combination_sampling(self):
        """Perform linear combination sampling from the sets S and H."""
        # Ensure H is not empty and contains valid trial points
        if len(self.H) == 0:
            raise ValueError("History H is empty; cannot perform linear combination sampling.")
        
        # Combine S and a sample of trials from H
        sample_size = max(1, int(self.beta_h * len(self.H)))  # Ensure at least one sample is taken
        sampled_H = np.random.choice(range(len(self.H)), size=sample_size, replace=False)
        S_prime = self.S + [self.H[i] for i in sampled_H]

        # Randomly select two points from S_prime, which is now a list of numpy arrays
        xa, xb = np.random.choice(len(S_prime), size=2, replace=False)
        xa, xb = S_prime[xa], S_prime[xb]

        # Generate a random weight for the linear combination
        alpha = np.random.normal(self.mu, self.sigma)

        # Perform the linear combination
        c = alpha * xa + (1 - alpha) * xb
        return c

    def ball_sampling(self):
        """Perform ball sampling around the best point in S."""
        # Find the best point in S
        best_point = min(self.S, key=lambda x: self.evaluate(x))
        
        # Sample a radius from a 1/r distribution
        r = np.random.exponential(scale=1.0)
        
        # Sample a random direction on the unit sphere
        direction = np.random.randn(self.dimension)
        direction /= np.linalg.norm(direction)
        
        # Create a new point
        c = best_point + r * direction
        return c
    
    def update_best_set(self, c):
        """Update the set of best trials S with the new point c if it improves."""
        f_c = self.evaluate(c)

        # Check if we need to add the new candidate point
        if len(self.S) < self.K_best:
            self.S.append(c)
        else:
            # Evaluate all points in S and find the worst one
            worst_index = np.argmax([self.evaluate(point) for point in self.S])
            worst_value = self.evaluate(self.S[worst_index])
            
            if f_c < worst_value:
                # Remove the worst point using its index
                self.S.pop(worst_index)  # Remove the worst point
                self.S.append(c)  # Add the new point

    def optimize(self, max_trials=1000):
        """
        Run the optimization loop.
        :param max_trials: Maximum number of trials to perform.
        """
        for t in range(max_trials):
            # Choose a sampling strategy
            if np.random.rand() < self.pl and len(self.H) >= 2:
                c = self.linear_combination_sampling()
            elif np.random.rand() < self.pb / (1 - self.pl) and len(self.H) >= 1:
                c = self.ball_sampling()
            else:
                c = self.sample_random()

            # Check if the point is within the feasible range
            if np.any(np.abs(c) > 5):
                continue  # Skip infeasible points

            # Add the new point to history
            self.H.append(c)

            # Update the set of best trials
            self.update_best_set(c)
            
        # Return the best solution found
        best_point = min(self.S, key=self.evaluate)
        return best_point, self.evaluate(best_point)

# Example usage
objective = benchmarks.ackley_function  # You can replace this with any of the benchmark functions
optimizer = LinearCombinationSwarm(objective, dimension=5)

best_solution, best_value = optimizer.optimize(max_trials=1000)
print("Best Solution:", best_solution)
print("Best Value:", best_value)

Best Solution: [ 0.08633861  0.132333    0.25696568  0.08063397 -0.06885176]
Best Value: 1.3594396601210694
